In [12]:
# install
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Libraries
import requests
import json
from tqdm import tqdm
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
# GLOBAL VARIABLES
country = 'MLM'
category = 'MLM1648'  # example_categories.json
start_offset = 0
end_offset = 100  # 1000 top items

In [17]:
def fetch_paginated_data_to_df_filtered(category, country, start_offset, end_offset):
    base_url = "https://api.mercadolibre.com/sites/"
    all_data = []
    
    for offset in tqdm(range(start_offset, end_offset + 1, 50), desc="Fetching paginated data"):
        # Construct the URL with the given parameters
        url = f"{base_url}{country}/search?category={category}&offset={offset}"
        
        # Make the API call
        response = requests.get(url)
        
        # Check if the call was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Extract the required fields
            for item in data.get('results', []):
                filtered_item = {
                    'id': item.get('id'),
                    'seller_id': item.get('seller', {}).get('id'),
                    'price': item.get('price'),
                    'original_price': item.get('original_price'),
                    'available_quantity': item.get('available_quantity'),
                }
                all_data.append(filtered_item)
        else:
            print(f"Error in the API call at offset {offset}")
    
    # Create a DataFrame from the filtered data
    df_filtered = pd.DataFrame(all_data)
    
    return df_filtered


# Call the function
df_filtered_paginated_data = fetch_paginated_data_to_df_filtered(category, country, start_offset, end_offset)
print(df_filtered_paginated_data.head())

Fetching paginated data: 100%|██████████| 21/21 [00:11<00:00,  1.88it/s]

              id   seller_id   price  original_price  available_quantity
0  MLM2023658727   581496608  6299.0          6499.0                   1
1  MLM3374476492   276696598  2185.5          2350.0                   1
2  MLM1988713103   226900267  2899.0             NaN                 500
3  MLM3414839820  1331658732  3699.0             NaN                 500
4   MLM931980768   414112680   349.0             NaN                 500


### Items metrics

In [11]:
def get_item_info(item_ids):
    all_response = []

    for item_id in tqdm(item_ids, desc="Fetching item info"):
        url = f"https://api.mercadolibre.com/items/{item_id}"
        response = requests.get(url)

        if response.status_code == 200:
            item_data = response.json()
            filtered_item = {
                'id': item_data.get('id'),
                'seller_id': item_data.get('seller_id'),
                'price': item_data.get('price'),
                'base_price': item_data.get('base_price'),
                'original_price': item_data.get('original_price'),
                'initial_quantity': item_data.get('initial_quantity'),
                'available_quantity': item_data.get('available_quantity'),
                'sold_quantity': item_data.get('sold_quantity'),
                'health': item_data.get('health'),
                'date_created': item_data.get('date_created'),
                'last_updated': item_data.get('last_updated')
            }
            all_response.append(filtered_item)
        else:
            print(f"Error: {response.status_code}")
    
    # Convert the response to a DataFrame
    df_item_info = pd.DataFrame(all_response)
        
    return df_item_info

# Example: Replace 'ITEM_ID' with a real ID
item_ids = df_filtered_paginated_data['id'].unique().tolist()
df_item_info = get_item_info(item_ids)

if df_item_info is not None:
    print(df_item_info.head())


Fetching item info:   0%|          | 0/997 [00:00<?, ?it/s]

Fetching item info: 100%|██████████| 997/997 [03:02<00:00,  5.47it/s]

              id   seller_id   price  base_price  original_price  \
0  MLM2023658727   581496608  6499.0      6499.0          6499.0   
1  MLM3374476492   276696598  2350.0      2350.0             NaN   
2  MLM1988713103   226900267  2899.0      2899.0             NaN   
3  MLM3414839820  1331658732  3699.0      3699.0             NaN   
4   MLM931980768   414112680   349.0       349.0             NaN   

   initial_quantity available_quantity sold_quantity  health  \
0               399               None          None     NaN   
1               916               None          None     1.0   
2              3010               None          None     NaN   
3              1123               None          None     NaN   
4             43509               None          None     NaN   

               date_created              last_updated  
0  2024-02-22T17:20:34.000Z  2024-12-10T04:07:41.965Z  
1  2024-09-23T02:18:59.069Z  2024-12-10T04:11:49.318Z  
2  2023-12-05T23:44:32.000Z  2024-12-1

In [18]:
# Perform the join of df_filtered_paginated_data with df_item_info using the 'id' column
df_items = pd.merge(df_filtered_paginated_data, df_item_info, on='id', suffixes=('_filtered', '_item_info'))

# Display the first rows of the resulting DataFrame
print(df_items.head())

              id  seller_id_filtered  price_filtered  original_price_filtered  \
0  MLM2023658727           581496608          6299.0                   6499.0   
1  MLM3374476492           276696598          2185.5                   2350.0   
2  MLM1988713103           226900267          2899.0                      NaN   
3  MLM3414839820          1331658732          3699.0                      NaN   
4   MLM931980768           414112680           349.0                      NaN   

   available_quantity_filtered  seller_id_item_info  price_item_info  \
0                            1            581496608           6499.0   
1                            1            276696598           2350.0   
2                          500            226900267           2899.0   
3                          500           1331658732           3699.0   
4                          500            414112680            349.0   

   base_price  original_price_item_info  initial_quantity  \
0      6499.0      

In [23]:
# Create a new column indicating if the original price is available
df_items['has_original_price'] = df_items['original_price_filtered'].notnull().astype(int)

# Create a new column calculating the discount percentage
df_items['discount_percentage'] = ((df_items['original_price_filtered'] - df_items['price_filtered']) / df_items['original_price_filtered']) * 100

# Fill NaN values in the discount column with 0 (no discount)
df_items['discount_percentage'].fillna(0, inplace=True)

# Create a new column indicating if the item has health information
df_items['has_health_info'] = df_items['health'].notnull().astype(int)

# Create a new column calculating the age of the item in days since it was created
df_items['item_age_days'] = (pd.to_datetime('today').tz_localize(None) - pd.to_datetime(df_items['date_created']).dt.tz_localize(None)).dt.days

print(df_items.head())

              id  seller_id_filtered  price_filtered  original_price_filtered  \
0  MLM2023658727           581496608          6299.0                   6499.0   
1  MLM3374476492           276696598          2185.5                   2350.0   
2  MLM1988713103           226900267          2899.0                      NaN   
3  MLM3414839820          1331658732          3699.0                      NaN   
4   MLM931980768           414112680           349.0                      NaN   

   available_quantity_filtered  seller_id_item_info  price_item_info  \
0                            1            581496608           6499.0   
1                            1            276696598           2350.0   
2                          500            226900267           2899.0   
3                          500           1331658732           3699.0   
4                          500            414112680            349.0   

   base_price  original_price_item_info  initial_quantity  \
0      6499.0      

In [25]:
df_items.columns

Index(['id', 'seller_id_filtered', 'price_filtered', 'original_price_filtered',
       'available_quantity_filtered', 'seller_id_item_info', 'price_item_info',
       'base_price', 'original_price_item_info', 'initial_quantity',
       'available_quantity_item_info', 'sold_quantity', 'health',
       'date_created', 'last_updated', 'has_original_price',
       'discount_percentage', 'has_health_info', 'item_age_days'],
      dtype='object')

In [27]:
# Group by seller_id and calculate metrics
seller_metrics = df_items.groupby('seller_id_filtered').agg({
    'id': 'count',  # Number of items
    'price_filtered': ['mean', 'min', 'max'],  # Average, minimum, and maximum price
    'original_price_filtered': ['mean', 'min', 'max'],  # Average, minimum, and maximum original price
    'available_quantity_filtered': 'sum',  # Total available quantity
    'discount_percentage': ['mean', 'min', 'max'],  # Average, minimum, and maximum discount percentage
    'health': ['mean', 'min', 'max'],  # Average, minimum, and maximum health
    'item_age_days': ['mean', 'min', 'max']  # Average, minimum, and maximum item age in days
}).reset_index()

# Rename columns for better readability
seller_metrics.columns = ['seller_id', 'item_count', 'avg_price', 'min_price', 'max_price', 
                          'avg_original_price', 'min_original_price', 'max_original_price', 
                          'total_available_quantity', 'avg_discount_percentage', 
                          'min_discount_percentage', 'max_discount_percentage',
                          'avg_health', 'min_health', 'max_health', 
                          'avg_item_age_days', 'min_item_age_days', 'max_item_age_days']

print(seller_metrics.head())

   seller_id  item_count  avg_price  min_price  max_price  avg_original_price  \
0    4881115           1   249.0000     249.00     249.00                 NaN   
1   10265533           2   496.9950     495.00     498.99                 NaN   
2   23299053           1   470.2500     470.25     470.25             495.000   
3   23838355          32  9589.5625    5679.29   19599.00           13270.875   
4   25926835           1    64.9900      64.99      64.99                 NaN   

   min_original_price  max_original_price  total_available_quantity  \
0                 NaN                 NaN                       500   
1                 NaN                 NaN                         2   
2               495.0               495.0                         1   
3              7999.0             26999.0                      1419   
4                 NaN                 NaN                        50   

   avg_discount_percentage  min_discount_percentage  max_discount_percentage  \
0     

### Seller data

In [28]:
def get_seller_info(seller_ids):
    all_seller_data = []

    for seller_id in tqdm(seller_ids, desc="Fetching seller info"):
        url = f"https://api.mercadolibre.com/users/{seller_id}"
        response = requests.get(url)

        if response.status_code == 200:
            seller_data = response.json()
            filtered_seller_data = {
                'id': seller_data.get('id'),
                'nickname': seller_data.get('nickname'),
                'user_type': seller_data.get('user_type'),
                'site_id': seller_data.get('site_id'),
                'permalink': seller_data.get('permalink'),
                'level_id': seller_data.get('seller_reputation', {}).get('level_id'),
                'power_seller_status': seller_data.get('seller_reputation', {}).get('power_seller_status'),
                'transactions_total': seller_data.get('seller_reputation', {}).get('transactions', {}).get('total'),
                'status': seller_data.get('status', {}).get('site_status')
            }
            all_seller_data.append(filtered_seller_data)
        else:
            print(f"Error: {response.status_code} for seller_id {seller_id}")

    # Convert the list of seller data to a DataFrame
    df_seller_info = pd.DataFrame(all_seller_data)
    
    return df_seller_info

# Example usage: Replace 'SELLER_ID_LIST' with a real list of seller IDs
seller_ids = seller_metrics['seller_id'].unique().tolist()
df_seller_info = get_seller_info(seller_ids)

if df_seller_info is not None:
    print(df_seller_info.head())

Fetching seller info: 100%|██████████| 426/426 [01:17<00:00,  5.53it/s]


         id                nickname user_type site_id  \
0   4881115             ZUGARS GAME    normal     MLM   
1  10265533                  RYSCOM     brand     MLM   
2  23299053                  DALKOM    normal     MLM   
3  23838355              GRUPODECME     brand     MLM   
4  25926835  RIVA IMPORTACIONES.GDL    normal     MLM   

                                           permalink level_id  \
0      http://perfil.mercadolibre.com.mx/ZUGARS+GAME  5_green   
1           http://perfil.mercadolibre.com.mx/RYSCOM  5_green   
2           http://perfil.mercadolibre.com.mx/DALKOM  5_green   
3       http://perfil.mercadolibre.com.mx/GRUPODECME  5_green   
4  http://perfil.mercadolibre.com.mx/RIVA+IMPORTA...  5_green   

  power_seller_status  transactions_total  status  
0            platinum               43036  active  
1            platinum               10853  active  
2            platinum                7448  active  
3            platinum              230004  active  
4     

In [29]:
# Perform the join of df_seller_info with seller_metrics using the 'seller_id' column
df_seller_final = pd.merge(df_seller_info, seller_metrics, left_on='id', right_on='seller_id')

# Display the first rows of the resulting DataFrame
print(df_seller_final.head())

         id                nickname user_type site_id  \
0   4881115             ZUGARS GAME    normal     MLM   
1  10265533                  RYSCOM     brand     MLM   
2  23299053                  DALKOM    normal     MLM   
3  23838355              GRUPODECME     brand     MLM   
4  25926835  RIVA IMPORTACIONES.GDL    normal     MLM   

                                           permalink level_id  \
0      http://perfil.mercadolibre.com.mx/ZUGARS+GAME  5_green   
1           http://perfil.mercadolibre.com.mx/RYSCOM  5_green   
2           http://perfil.mercadolibre.com.mx/DALKOM  5_green   
3       http://perfil.mercadolibre.com.mx/GRUPODECME  5_green   
4  http://perfil.mercadolibre.com.mx/RIVA+IMPORTA...  5_green   

  power_seller_status  transactions_total  status  seller_id  ...  \
0            platinum               43036  active    4881115  ...   
1            platinum               10853  active   10265533  ...   
2            platinum                7448  active   2329905